In [1]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image 
import random
import os
import shutil
from itertools import chain, product
import random
from time import time

In [2]:
t1 = time()

## 1- Récupération des data

* C, NC, empty -> images avec languette conforme, languette non conforme, sans languette
* tabs/C, /NC, /empty -> fenêtres avec languette conforme, languette non conforme, sans languette
* synthetic/... -> images synthétiques

In [3]:
C = r"D:/data/C"
NC = r"D:/data/NC"
EMPTY = r"D:/data/empty"
TABS_EMPTY = r"D:/data/tabs/empty"
TABS_C = r"D:/data/tabs/C"
TABS_NC = r"D:/data/tabs/NC"
SYNTHETIC_NC = r"D:/data/synthetic/NC"
SYNTHETIC_C = r"D:/data/synthetic/C"
SYNTHETIC_EMPTY = r"D:/data/synthetic/empty"
SEGMENTED_TABS_C = r"D:/data\tabs\segmented_tabs\C"
SEGMENTED_TABS_NC = r"D:/data\tabs\segmented_tabs\NC"
# fenêtre languette
x0, y0, x1, y1 = 0, 25, 60, 65

Récupération d'images conformes

In [4]:
for img in glob.glob(r"D:\train_double_tape_LH03\C\*.bmp")[0:10]:
    shutil.copy(img, os.path.join(C, os.path.basename(img)))

Récupération d'images sans languette

Récupération d'images avec languette non conforme

In [5]:
for img in glob.glob(r"D:\train_double_tape_LH03\NC\*.bmp")[0:10]:
    shutil.copy(img, os.path.join(NC,os.path.basename(img)))

Récupération de fenêtres avec languette absente

In [6]:
# récupération d'une fenêtre de l'image où se trouve la portion à extraire
def extract_window(image_path, x0, y0, x1, y1) -> 'np.ndarray':   
    # Vérification du chemin
    assert os.path.exists(image_path), "Le fichier n'existe pas au chemin spécifié"
    
    # Chargement de l'image
    image = cv2.imread(image_path)
    assert image is not None, "L'image n'a pas pu être lue"
    
    # Recadrage
    img_cropped = image[x0:x1, y0:y1]                     # ATTENTION à l'ordre des coordonnées
    
    # Affichage de l'image 
    #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    #plt.title("Image originale")
    #plt.show()
    
    # Affichage de l'image recadrée
    #plt.imshow(cv2.cvtColor(img_cropped, cv2.COLOR_BGR2RGB))
    #plt.title("Image recadrée")
    #plt.show()

    return img_cropped

# exemple :
image_path = r"data\NC\LH_11371_03_OK.bmp"
#extract_window(image_path, x0, y0, x1, y1)

In [7]:
for img in glob.glob(os.path.join(EMPTY,"*.bmp")):
   cv2.imwrite(os.path.join(TABS_EMPTY, os.path.basename(img)), extract_window(img, x0, y0, x1, y1))

Récupération d'images de languette C et NC

In [8]:
for img in glob.glob(os.path.join(C,"*.bmp"))[0:10]:
   cv2.imwrite(os.path.join(TABS_C,os.path.basename(img)), extract_window(img, x0, y0, x1, y1))

for img in glob.glob(os.path.join(NC,"*.bmp"))[0:10]:
   cv2.imwrite(os.path.join(TABS_NC,os.path.basename(img)), extract_window(img, x0, y0, x1, y1))

Segmentation des images de languette NC

In [9]:
for img in glob.glob(os.path.join(TABS_NC, "*.bmp")):
    image = cv2.imread(img)
    
    # Ajuster le rectangle pour couvrir toute l'image
    height, width = image.shape[:2]
    rectangle = (1, 1, width - 5, height - 2)  # Presque toute l'image, en évitant les bords
    
    # Réinitialiser le masque et les modèles
    mask = np.zeros(image.shape[:2], np.uint8)
    backgroundModel = np.zeros((1, 65), np.float64)
    foregroundModel = np.zeros((1, 65), np.float64)
    
    # Appliquer GrabCut avec le rectangle ajusté
    cv2.grabCut(image, mask, rectangle, backgroundModel, foregroundModel, 5, cv2.GC_INIT_WITH_RECT)
    
    # Visualiser le masque généré
    #plt.figure(figsize=(10, 5))
    #plt.subplot(1, 3, 1)
    #plt.title('Masque initial (GrabCut)')
    #plt.imshow(mask, cmap='gray')
    #plt.axis('off')
    
    # Mettre à jour le masque pour extraire le premier plan
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    
    # Appliquer le masque à l'image
    image_segmented = image * mask2[:, :, np.newaxis]

    # centrage des images segmentées sur la queue des languettes
    height, width, channels = image_segmented.shape
    new_height = int(height * 5 / 3)
    augmented_image = np.zeros((new_height, width, channels), dtype=image.dtype)
    augmented_image[0:height, 0: width, :] = image_segmented
    cv2.imwrite(os.path.join(SEGMENTED_TABS_NC, os.path.basename(img)), augmented_image)
    
    # Afficher le résultat
    #plt.subplot(1, 3, 2)
    #plt.title('Image Segmentée')
    #plt.imshow(augmented_image)
    #plt.axis('off')
    
    #plt.subplot(1, 3, 3)
    #plt.title('Image Originale')
    #plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    #plt.axis('off')
    
    #plt.tight_layout()
    #plt.show()

## 2- Génération d'images synthétiques sans languette

* Par combinaison d'images vides
* Par remplacement de la zone d'intérêt d'images C ou NC par une fenêtre vide

Par combinaison d'images vides

In [10]:
i = 0
for img1, img2 in product(glob.glob(os.path.join(EMPTY,"*.bmp")), glob.glob(os.path.join(EMPTY,"*.bmp"))):
    
    # Normalisation ou transformation des images (division par 2 ici)
    img1 = (cv2.imread(img1) // 2).astype('uint8')
    img2 = (cv2.imread(img2) // 2).astype('uint8')
    
    # Additionner les images
    result = img1 + img2 
    
    # Afficher le résultat
    #plt.imshow(result)
    #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_EMPTY, "synth_image_1_{i}.bmp"), result)

    i += 1

Par remplacement de la zone d'intérêt d'images C ou NC par une fenêtre vide

In [11]:
i = 0
for b, w in product(glob.glob(os.path.join(NC, "*.bmp")), glob.glob(os.path.join(TABS_EMPTY, "*.bmp"))):
    
    b = cv2.imread(b)
    w = cv2.imread(w)
    #plt.imshow(b)  
    #plt.title('b_before')
    #plt.show()
    
    # adaptation de la luminosité de la fenêtre à son voisinage

    # Sélectionner une région de `w` pour le calcul de la luminosité
    region_w = w[int(w.shape[0] * 1 / 3):int(w.shape[0] * 2 / 3), int(w.shape[1] * 2.5 / 3):int(w.shape[1] * 3 / 3), :]
    lum_w = np.mean(region_w)
    
    # Sélectionner une région de `b` pour le calcul de la luminosité
    region_b = b[x0:x1, y0:y1, :]
    lum_b = np.mean(region_b[int(region_b.shape[0] * 1 / 3):int(region_b.shape[0] * 2 / 3), int(region_b.shape[1] * 2.5 / 3):int(region_b.shape[1] * 3 / 3), :])
    
    # Différence de luminosité
    delta_lum = lum_b - lum_w
    # print(f"Luminosité moyenne - w: {lum_w}, b: {lum_b}, delta: {delta_lum}")
    
    # Ajuster la luminosité de `w`
    w_float = w.astype(np.float32)  # Conversion en float pour éviter les saturations
    w_adjusted = w_float * (lum_b / lum_w)  # Ajustement proportionnel
    
    # Clip pour éviter les valeurs hors plage
    w_adjusted = np.clip(w_adjusted, 0, 255).astype(np.uint8)
    #plt.imshow(w)
    #plt.title('w')
    #plt.show()
    
    #plt.imshow(w_adjusted)
    #plt.title('w_adjusted')
    #plt.show()
    
    b[x0:x1, y0:y1] = w_adjusted # ATTENTION à la position de collage
    #plt.imshow(b)
    #plt.title('b_after')
    #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_EMPTY,"synth_2_{i}.bmp"), b)
    i += 1

## 3- Génération d'images synthétiques de défauts

* Par superposition de languettes NC sur des images vides => synth_NC_a_
* Par superposition de languette conforme décalée sur des images vides => synth_NC_b_
* Par superposition de languette conforme décalée sur des images conformes => synth_NC_c_
* Par superposition de deux languettes conformes décalées sur des images vides => synth_NC_d_

Par superposition de languettes NC sur des images vides

In [12]:
# Init shift range
s_x0 = 2
s_x1 = 4
s_y0 = 2
s_y1 = 4
s_a0 = 5
s_a1 = 10

In [13]:
i = 0
for bgd_path, w_path in product(glob.glob(os.path.join(EMPTY, "*.bmp")), glob.glob(os.path.join(SEGMENTED_TABS_NC, "*.bmp"))):
    # SYNTHETIC_EMPTY
    
    bgd = cv2.imread(bgd_path)
    w = cv2.imread(w_path)
    
    # Identifier les pixels noirs dans `w` (proches de [0, 0, 0])
    mask_black = np.all(w < [50, 50, 50], axis=-1)  # Masque des pixels noirs

    # Remplacer les pixels noirs par [0, 0, 0]
    w[mask_black] = [0, 0, 0]

    # Coller `w` dans la région spécifiée de `bgd`  
    bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]] = cv2.add(bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]], w)   
   
    #plt.imshow(bgd)
    #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_NC, f"synth_NC_a_{i}.bmp"), bgd)
    
    i += 1

Par superposition de languette conforme décalée sur des images vides

In [14]:
i = 0
for bgd_path, w_path in product(glob.glob(os.path.join(EMPTY, "*.bmp")), glob.glob(os.path.join(SEGMENTED_TABS_C, "*.bmp"))):
    # SYNTHETIC_EMPTY
    
    bgd = cv2.imread(bgd_path)
    w = Image.open(w_path)
    
    # Faire subir une rotation et une translation à languette
    s_x = random.choice([random.randrange(-s_x1, -s_x0), random.randrange(s_x0, s_x1)])
    s_y = random.choice([random.randrange(-s_y1, -s_y0), random.randrange(s_y0, s_y1)])
    s_a = random.choice([random.randrange(-s_a1, -s_a0), random.randrange(s_a0, s_a1)])
    
    w = w.rotate(s_a, translate=(s_x, s_y))
    
    # Identifier les pixels noirs dans `w` (proches de [0, 0, 0])
    w = np.array(w)
    mask_black = np.all(w < [50, 50, 50], axis=-1)  # Masque des pixels noirs

    # Remplacer les pixels noirs par [0, 0, 0]
    w[mask_black] = [0, 0, 0]

    # Coller `w` dans la région spécifiée de `bgd`  
    bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]] = cv2.add(bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]], w)   
   
    #plt.imshow(bgd)
    #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_NC, f"synth_NC_b_{i}.bmp"), bgd)
    
    i += 1

Par superposition de languette conforme décalée sur des images conformes

In [15]:
i = 0
for bgd_path, w_path in product(glob.glob(os.path.join(C, "*.bmp")), glob.glob(os.path.join(SEGMENTED_TABS_C, "*.bmp"))):
    # SYNTHETIC_EMPTY
    
    bgd = cv2.imread(bgd_path)
    w = Image.open(w_path)
    
    # Faire subir une rotation et une translation à languette
    s_x = random.choice([random.randrange(-s_x1, -s_x0), random.randrange(s_x0, s_x1)])
    s_y = random.choice([random.randrange(-s_y1, -s_y0), random.randrange(s_y0, s_y1)])
    s_a = random.choice([random.randrange(-s_a1, -s_a0), random.randrange(s_a0, s_a1)])
    
    w = w.rotate(s_a, translate=(s_x, s_y))
    
    # Identifier les pixels noirs dans `w` (proches de [0, 0, 0])
    w = np.array(w)
    mask_black = np.all(w < [50, 50, 50], axis=-1)  # Masque des pixels noirs

    # Remplacer les pixels noirs par [0, 0, 0]
    w[mask_black] = [0, 0, 0]

    # Coller `w` dans la région spécifiée de `bgd`  
    bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]] = cv2.add(bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]], w)   
   
    #plt.imshow(bgd)
    #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_NC, f"synth_NC_c_{i}.bmp"), bgd)
    
    i += 1

Par superposition de deux languettes conformes décalées sur des images vides

In [16]:
i = 0
for bgd_path, w_path in product(glob.glob(os.path.join(EMPTY, "*.bmp")), glob.glob(os.path.join(SEGMENTED_TABS_C, "*.bmp"))):
    # SYNTHETIC_EMPTY
    
    bgd = cv2.imread(bgd_path)
    w = Image.open(w_path)
    
    for k in [1,2]:
        # Faire subir une rotation et une translation à languette
        s_x = random.choice([random.randrange(-s_x1, -s_x0), random.randrange(s_x0, s_x1)])
        s_y = random.choice([random.randrange(-s_y1, -s_y0), random.randrange(s_y0, s_y1)])
        s_a = random.choice([random.randrange(-s_a1, -s_a0), random.randrange(s_a0, s_a1)])
        
        w = w.rotate(s_a, translate=(s_x, s_y))
        
        # Identifier les pixels noirs dans `w` (proches de [0, 0, 0])
        w = np.array(w)
        mask_black = np.all(w < [50, 50, 50], axis=-1)  # Masque des pixels noirs
    
        # Remplacer les pixels noirs par [0, 0, 0]
        w[mask_black] = [0, 0, 0]
    
        # Coller `w` dans la région spécifiée de `bgd`  
        bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]] = cv2.add(bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]], w)   

        w = Image.fromarray(w)
        #plt.imshow(bgd)
        #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_NC, f"synth_NC_d_{i}.bmp"), bgd)
    
    i += 1

# 4- Génération d'images synthétiques conformes

In [17]:
i = 0
for bgd_path, w_path in product(glob.glob(os.path.join(EMPTY, "*.bmp")), glob.glob(os.path.join(SEGMENTED_TABS_C, "*.bmp"))):
    # SYNTHETIC_EMPTY
    
    bgd = cv2.imread(bgd_path)
    w = cv2.imread(w_path)
    
    # Identifier les pixels noirs dans `w` (proches de [0, 0, 0])
    mask_black = np.all(w < [50, 50, 50], axis=-1)  # Masque des pixels noirs

    # Remplacer les pixels noirs par [0, 0, 0]
    w[mask_black] = [0, 0, 0]

    # Coller `w` dans la région spécifiée de `bgd`  
    bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]] = cv2.add(bgd[x0:x0+w.shape[0], y0:y0+w.shape[1]], w)   
   
    #plt.imshow(bgd)
    #plt.show()

    cv2.imwrite(os.path.join(SYNTHETIC_C, f"synth_C_{i}.bmp"), bgd)
    
    i += 1

In [18]:
time()-t1

55.098100900650024